# Algoritmo para el proyecto Densidad de Homomorfismos en Árboles. 

En este cuaderno se pretende mostrar el algoritmo descrito en la sección $7$ del documento adjunto. Como se menciona allí, la idea es implementar el modelo de programación lineal descrito en la sección $4$ para demostrar desigualdades relacionadas con la densidad de homomorfismos entre grafos. El algoritmo consta de tres partes, como se muestra a continuación:

Parte 1:
Creación de la matriz de coeficientes relacionada con las restricciones del modelo.

Esta primera parte del algoritmo inicia tomando como entrada dos árboles $F$ y $T$ y regresando todos los posibles homomorfismos entre estos árboles. El input, es decir Los árboles $F$ y $T$, se introducen como una lista de listas: $[l_1,l_2,...,l_{|V(F)|}]$, dónde la lista $l_i$ tiene como elementos todos los vecinos del vértice $i+1$. El output, es una lista que tiene todos los posiblles homomorfismos entre $F$ y $T$; cada homomorfismo se representa como una lista $[u_1,u_2,...,u_{|V(F)|}]$ tal que $u_i=f(i)$ para todo $i\in V(F)$.

In [1]:
def hom(F,T):
    L = []
    f = len(F)
    t = len(T)
    for i in range(t):
        L.append([i+1])
    for i in range(2,f+1):
        m = []
        for j in range(len(L)):
            s = min(F[i-1])
            for k in range(len(T[L[j][s-1]-1])):
                m.append(L[j] + [T[L[j][s-1]-1][k]])
        L = m
    return(L)

Debido a que la función *hom* está pensada para arrojar homomorfimos entre árboles, es necesario que tanto $F$ como $T$ estén numerados como se describe en la sección $7$ item $2$; la importancia de este hecho se ve en la escogencia del valor de $s$. A continuación vemos un ejemplo si tomamos $F=P_4$ y $T=P_3$. Estos homomorfismos fueron listados en la sección $5$ del documento para ilustrar cómo se puede utilizar el modelo de programación lineal para construir contraejemplos.

In [2]:
print(hom([[2],[1,3],[2,4],[3]],[[2],[1,3],[2]]))

[[1, 2, 1, 2], [1, 2, 3, 2], [2, 1, 2, 1], [2, 1, 2, 3], [2, 3, 2, 1], [2, 3, 2, 3], [3, 2, 1, 2], [3, 2, 3, 2]]


La siguiente parte del algoritmo consiste en crear la matriz que guarda la información de los coeficientes relacionados a las restricciones del problema lineal. Para esto, se hace uso de la ecuación $17$ del documento. Observe que la función *matrix* utiliza la función *hom* descrita anteriormente y, por cada homomorfismo, realiza el cálculo respectivo. Sin embargo, esta parte del algoritmo podría mejorarse si se observa que hay homomorfismos simétricos entre sí y con los cuales se obtendrá los mismos coeficientes. 

In [3]:
def matrix(F,T):
    D = hom(F,T)
    t = len(T)
    d = len(D)
    H = []
    E = []
    for i in range(t):
        for j in range(len(T[i])):
            if T[i][j] > i+1:
                E.append([i+1,T[i][j]])
    e = len(E)
    for i in range(e+t):
        H.append([])
        for j in range(d):
            H[i].append(0)
        if i <= e-1:
            H[i].append(-1)
        else:
            H[i].append(0)
    H.append([])
    H.append([])
    for j in range(d):
        H[-2].append(1)
        H[-1].append(-1)
    H[-2].append(0)
    H[-1].append(0)
    for i in range(d):
        finv = []
        for j in range(1,t+1):
            finv.append([i+1 for i, x in enumerate(D[i]) if x == j])
        for x in E:
            for y in finv[x[0]-1]:
                for z in finv[x[1]-1]:
                    if z in F[y-1]:
                        H[E.index(x)][i] += 1
        for j in range(t):
            for k in finv[j]:
                H[e+j][i] -= len(F[k-1])-1
    return([H,D])

La función *matrix* recibe como input dos árboles, al igual que *hom* y el output es una lista con dos elementos: el primer elemento es la matriz que buscamos y, el segundo elemento, es el resultado de la función *hom*. Esto es para evitar correr la función *hom* dos veces en los algoritmos que se presentan a continuación. Como ejemplo, veamos el resultado al tomar $F=P_4$ y $T=P_3$

In [5]:
print(matrix([[2],[1,3],[2,4],[3]],[[2],[1,3],[2]])[0])

[[3, 1, 3, 2, 1, 0, 2, 0, -1], [0, 2, 0, 1, 2, 3, 1, 3, -1], [-1, 0, -1, -1, 0, 0, -1, 0, 0], [-1, -1, -1, -1, -1, -1, -1, -1, 0], [0, -1, 0, 0, -1, -1, 0, -1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0], [-1, -1, -1, -1, -1, -1, -1, -1, 0]]


Parte 2: Organización y lectura apropiada de los árboles obtenidos de la base de datos https://users.cecs.anu.edu.au/~bdm/data/trees.html.

El primer código que se muestra a continuación permite leer la información obtenida en cada archivo de la base de datos. Desde ahí se puede descargar un archivo .rar que contiene varios archivos .tex con todos los posibles árboles de un tamaño determinado $n$. Así, la función *obt* requiere dos inputs: el tamaño del árbol y el número fil correspondiente al archivo que se desea leer. Todos los árboles del archivo fil se guardan en una lista, la cuál es el output de la función.

In [6]:
def obt(n,fil):
    with open(r"C:\Users\57319\OneDrive\Documentos\árboles\vtx" + str(n) + "s\\tree" + str(n) + str(fil) + ".txt") as f:
        c = f.readlines()
        a = []
        b = []
        for x in c:
            a.append(x.split("  "))
            b.append([])
        for i in range(len(a)):
            for x in a[i]:
                b[i].append(list(map(int,x.split(" "))))
    return(b)

Desafortunadamente, los árboles que se obtienen de los datos se encuentran expresados como una lista de duplas, donde cada dupla representa una arista del árbol. Recordemos que, para que se pueda aplicar la función *hom* y *matrix*, es necesario tener los árboles expresados como una lista que contiene, por cada vértice la lista de sus vecinos. Así, la siguiente función cambia el formato de los árboles que se obtuvieron con la función *obt*.

In [7]:
def obttrees(n,fil):
    a = obt(n,fil)
    b = []
    for x in a:
        s = []
        for i in range(n):
            s.append([])
        b.append(s)
        for edge in x:
            key, value = edge[0], edge[1]
            b[-1][key].append(value + 1)
            b[-1][value].append(key + 1)
    return(b)

Adicionalmente, se observa que los árboles que se obtienen de la base de datos se encuentran numerados de una forma distinta a la descrita en la sección $7$ item $2$ la cuál es importante no solo para poder aplicar la función *hom* apropiadamente, si no también para poder definir el sistema de programación lineal. Así, la siguiente función recibe un árbol con cualquier numeración y retorna el mismo árbol numerado apropiadamente.

In [8]:
def propnum(L):
    vtx = 2
    leng = len(L)
    for i in range(leng):
        for j in range(len(L[i])):
            x = L[i][j]
            if x > vtx:
                m = L[x-1]
                L[x-1] = L[vtx-1]
                L[vtx-1] = m
                for k in range(leng):
                    for l in range(len(L[k])):
                        if L[k][l] == x:
                            L[k][l] = vtx
                        elif L[k][l] == vtx:
                            L[k][l] = x
                vtx += 1
            elif x == vtx:
                vtx += 1
    return(L)

Como ejemplo de la función anterior, tomemos la estrella de $4$ vértices, es decir, el árbol formado por $4$ vértices, que tiene un vértice de grado $3$ y los demás vértices de grado $2$. Este árbol se encuentra representado en la base de datos como la lista $[[4],[4],[4],[1,2,3]]$. La numeración apropiada sería: $[[2,3,4],[1],[1],[1]]$ o, alternativamente, se puede usar la numeración $[[2],[1,3,4],[2],[2]]$. Recordemos que se desea que para todo $i>2$, el vértice $v_i$ tenga un único vecino en el conjunto $\{v_1,...,v_{i-1}\}$.

In [9]:
print(propnum([[4],[4],[4],[1,2,3]]))

[[2], [1, 3, 4], [2], [2]]


Parte 3: Uso de Gurobi para solucionar el problema de programación lineal.

Ya se tiene todo lo necesario para definir, solucionar y evaluar los resultados del modelo lineal asociado a este problema.

In [10]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

La siguiente función toma la matriz que se construyó con la función *matrix* y define el problema de optimización lineal en el lenguaje requerido por Gurobi. Además, en esta misma función se crea el vector b del sistema $Ax\geq b$. Finalmente, Gurobi guarda la información sobre la feasibilidad de un problema en la variable *m.status*; tomando ventaja de ello, la función imprime la matriz de *matrix* y retorna (0) si el modelo es infeasible y, por otro lado, imprime los homomorfismos que dieron solución al problema de minimización, los pesos asociados a estos homomorfismos y retorna (1) si el modelo es feasible.

In [26]:
def trees(F,T,ef,et):
    L = matrix(F,T)
    H = L[0]
    D = L[1]
    d = len(H[0])
    e = len(H)
    t = len(T)
    l = e-t-2
    m = gp.Model("matrix1")
    x = m.addMVar(shape=d, lb=0.0, ub=float(np.Infinity), vtype = GRB.CONTINUOUS, name="x")
    obj = np.array(np.zeros(d))
    obj[d-1] = 1
    m.setObjective(obj @ x, GRB.MINIMIZE)

    val = np.array(H).flatten()
    row = np.zeros(e*d)
    for i in range(e*d):
        row[i] = i // d
    col = np.zeros(e*d)
    for i in range(e*d):
        col[i] = i % d
    A = sp.csr_matrix (( val , (row, col)) , shape = (e,d))
    
    #print(A)
    rh = np.zeros(e)
    for i in range(l,e-2):
        rh[i] = (-ef/et)*(len(T[i-l])-1)
    rh[e-2] = 1
    rh[e-1] = -1
    #print(rh)
    rhs = np.array(rh)
    
    m.addConstr(A @ x <= rhs, name="c")
    m.optimize()
    
    op = m.status
    
    if(op == 3):
        print("Matriz: ", H)
        return(0)
    else:
        print("Pesos asociados a los homomorfismos: ",x.X)
        nnz = np.nonzero(x.X)[0].tolist()
        nnz.pop(-1)
        
        print("Homomorfismos utilizados: ")
        for x in nnz:
            print(D[x])
    
        return(1)

Por ejemplo, veamos que el modelo es infeasible si se toma $F=P_4$ y $T=P_3$.

In [27]:
print(trees([[2],[1,3],[2,4],[3]],[[2],[1,3],[2]],3,2))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 9 columns and 46 nonzeros
Model fingerprint: 0xd186e1f9
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model
Matriz:  [[3, 1, 3, 2, 1, 0, 2, 0, -1], [0, 2, 0, 1, 2, 3, 1, 3, -1], [-1, 0, -1, -1, 0, 0, -1, 0, 0], [-1, -1, -1, -1, -1, -1, -1, -1, 0], [0, -1, 0, 0, -1, -1, 0, -1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0], [-1, -1, -1, -1, -1, -1, -1, -1, 0]]
0


Adicionalmente, veamos que el modelo es feasible si se toma $F=P_5$ y $T=P_3$. Este caso se estudió en el Ejemplo $3.2$ del documento. El programa arroja homomorfismos distintos a los encontrados en el ejemplo.

In [28]:
print(trees([[2],[1,3],[2,4],[3,5],[4]],[[2],[1,3],[2]],4,2))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 13 columns and 72 nonzeros
Model fingerprint: 0x04c642df
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolved: 4 rows, 9 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.000000e+00   0.000000e+00      0s
       3    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.000000000e+00
Pesos asociados a los homomorfismos:  [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 2. ]
Homomorfismos utilizados: 
[1, 2, 1, 2, 1]
[3, 2, 3, 2, 3]
1


Cabe resaltar que la última entrada de la lista que muestra los pesos asociados a los homomorfismos es siempre el valor que toma la variable $t$ y el cuál debe coincidir con $\frac{|E(F)|}{|E(T)|}$.

Finalmente, la función *hold* toma los algoritmos descritos anteriormente y hace un recorrido por todos los posibles árboles con $n$ números de vértices para evaluar la feasibilidad del modelo relacionado a la desigualdad $t(T,G)\geq t(P_3,G)^{|E(T)|/2}$. Observe que esta función se puede modificar si se desa evaluar $t(T,G)\geq t(F,G)^{|E(T)|/|E(F)|}$ para cualquier árbol $F$. Esta función retorna una lista cuya primera entrada corresponde a los árboles que están relacionados con modelos infeasibles y la segunda entrada corresponde a árboles que están relacionados con modelos feasibles.

In [29]:
def hold(n):
    L = []
    P = []
    for i in range(2,n):
        G = obttrees(n,i)
        F = list(map(propnum,obttrees(n,i)))
        for tree in F:
            print("\nFor F = ", F)
            if trees(tree,[[2],[1,3],[2]],n-1,2) == 0:
                L.append(tree)
            else : 
                P.append(tree)
    return([L,P])

Veamos el resultado de evaluar $t(T,G)\geq t(P_3,G)^{|E(T)|/2}$ para todos los posibles árboles $T$ de hasta $10$ vértices.

In [32]:
hold(4)


For F =  [[[2], [1, 3, 4], [2], [2]]]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 11 columns and 48 nonzeros
Model fingerprint: 0xdf4e9822
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+00   0.000000e+00      0s
       4    1.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.500000000e+00
Pesos asociados a los homomorfismos:  [0.5  0.   0.   0.   0.   0.25 0.   0.   0.   0.25 1.5 ]
Homomorfismos utilizad

[[[[2, 3], [1], [1, 4], [3]]], [[[2], [1, 3, 4], [2], [2]]]]

In [31]:
hold(5)


For F =  [[[2], [1, 3, 4, 5], [2], [2], [2]]]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 19 columns and 88 nonzeros
Model fingerprint: 0x48d1b937
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 3 rows and 11 columns
Presolve time: 0.00s
Presolved: 4 rows, 8 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+00   0.000000e+00      0s
       4    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.000000000e+00
Pesos asociados a los homomorfismos:  [0.5        0.         0.         0.         0.         0.
 0.        

[[],
 [[[2], [1, 3, 4, 5], [2], [2], [2]],
  [[2, 3], [1], [1, 4, 5], [3], [3]],
  [[2, 3], [1], [1, 4], [5, 3], [4]]]]

In [33]:
hold(6)


For F =  [[[2], [1, 3, 4, 5, 6], [2], [2], [2], [2]]]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 35 columns and 168 nonzeros
Model fingerprint: 0xf119bc89
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 3 rows and 26 columns
Presolve time: 0.00s
Presolved: 4 rows, 9 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.250000e+00   0.000000e+00      0s
       4    2.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.500000000e+00
Pesos asociados a los homomorfismos:  [0.5   0.    0.    0.    0.    0.    0.    0.    0.    0.    

[[[[2], [1, 3, 4], [2], [5, 6, 2], [4], [4]],
  [[2, 3], [1], [1, 4, 5], [6, 3], [3], [4]],
  [[2, 3], [1], [1, 4], [5, 3], [4, 6], [5]]],
 [[[2], [1, 3, 4, 5, 6], [2], [2], [2], [2]],
  [[2, 3], [1], [1, 4, 5, 6], [3], [3], [3]],
  [[2, 3], [1, 4], [1, 5, 6], [2], [3], [3]]]]

In [34]:
hold(7)


For F =  [[[2], [1, 3, 4, 5, 6, 7], [2], [2], [2], [2], [2]]]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 67 columns and 328 nonzeros
Model fingerprint: 0xe97a69c4
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 3 rows and 57 columns
Presolve time: 0.00s
Presolved: 4 rows, 10 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.125000e+00   0.000000e+00      0s
       4    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.000000000e+00
Pesos asociados a los homomorfismos:  [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

[[],
 [[[2], [1, 3, 4, 5, 6, 7], [2], [2], [2], [2], [2]],
  [[2, 3], [1], [1, 4, 5, 6, 7], [3], [3], [3], [3]],
  [[2], [1, 3, 4], [2], [5, 6, 7, 2], [4], [4], [4]],
  [[2, 3], [1, 4], [1, 5, 6, 7], [2], [3], [3], [3]],
  [[2, 3], [1, 4, 5], [1, 6, 7], [2], [2], [3], [3]],
  [[2, 3], [1], [1, 4, 5, 6], [7, 3], [3], [3], [4]],
  [[2, 3], [1], [1, 4, 5], [3], [6, 7, 3], [5], [5]],
  [[2, 3], [1], [1, 4, 5], [6, 3], [7, 3], [4], [5]],
  [[2, 3], [1], [1, 4], [3, 5], [4, 6, 7], [5], [5]],
  [[2, 3], [1], [1, 4, 5], [6, 3], [3], [4, 7], [6]],
  [[2, 3], [1], [1, 4], [3, 5], [6, 4], [7, 5], [6]]]]

In [35]:
hold(8)


For F =  [[[2], [1, 3, 4, 5, 6, 7, 8], [2], [2], [2], [2], [2], [2]]]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 131 columns and 648 nonzeros
Model fingerprint: 0x8baaf181
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 3 rows and 120 columns
Presolve time: 0.00s
Presolved: 4 rows, 11 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.312500e+00   0.000000e+00      0s
       4    3.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.500000000e+00
Pesos asociados a los homomorfismos:  [0.5        0.         0.         0.      

[[[[2], [1, 3, 4, 5], [2], [2], [6, 7, 8, 2], [5], [5], [5]],
  [[2, 3], [1], [1, 4, 5, 6], [3], [3], [7, 8, 3], [6], [6]],
  [[2, 3], [1], [1, 4, 5, 6], [7, 3], [8, 3], [3], [4], [5]],
  [[2, 3], [1, 4], [1, 5, 6], [7, 8, 2], [3], [3], [4], [4]],
  [[2], [1, 3, 4], [2], [5, 6, 2], [7, 4], [4], [5, 8], [7]],
  [[2, 3, 4], [1], [1, 5], [1, 6, 7], [3], [8, 4], [4], [6]],
  [[2, 3], [1], [1, 4, 5], [6, 3], [7, 3], [4], [5, 8], [7]],
  [[2, 3], [1], [1, 4], [3, 5], [6, 4, 7], [8, 5], [5], [6]],
  [[2, 3], [1], [1, 4], [5, 3], [6, 4], [5, 7], [6, 8], [7]]],
 [[[2], [1, 3, 4, 5, 6, 7, 8], [2], [2], [2], [2], [2], [2]],
  [[2, 3], [1], [1, 4, 5, 6, 7, 8], [3], [3], [3], [3], [3]],
  [[2], [1, 3, 4], [2], [5, 6, 7, 8, 2], [4], [4], [4], [4]],
  [[2, 3], [1, 4], [1, 5, 6, 7, 8], [2], [3], [3], [3], [3]],
  [[2, 3], [1, 4, 5], [1, 6, 7, 8], [2], [2], [3], [3], [3]],
  [[2, 3], [1], [1, 4, 5, 6, 7], [8, 3], [3], [3], [3], [4]],
  [[2, 3, 4], [1], [1, 5], [1, 6, 7, 8], [3], [4], [4], [4]],
  [[2],

In [36]:
hold(9)


For F =  [[[2], [1, 3, 4, 5, 6, 7, 8, 9], [2], [2], [2], [2], [2], [2], [2]]]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 259 columns and 1288 nonzeros
Model fingerprint: 0x96e4d63a
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 3 rows and 247 columns
Presolve time: 0.00s
Presolved: 4 rows, 12 columns, 40 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+00   0.000000e+00      0s
       4    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.000000000e+00
Pesos asociados a los homomorfismos:  [0.5        0.         0.        

[[],
 [[[2], [1, 3, 4, 5, 6, 7, 8, 9], [2], [2], [2], [2], [2], [2], [2]],
  [[2, 3], [1], [1, 4, 5, 6, 7, 8, 9], [3], [3], [3], [3], [3], [3]],
  [[2], [1, 3, 4], [2], [5, 6, 7, 8, 9, 2], [4], [4], [4], [4], [4]],
  [[2], [1, 3, 4, 5], [2], [2], [6, 7, 8, 9, 2], [5], [5], [5], [5]],
  [[2, 3], [1, 4], [1, 5, 6, 7, 8, 9], [2], [3], [3], [3], [3], [3]],
  [[2, 3], [1, 4, 5], [1, 6, 7, 8, 9], [2], [2], [3], [3], [3], [3]],
  [[2, 3], [1, 4, 5, 6], [1, 7, 8, 9], [2], [2], [2], [3], [3], [3]],
  [[2, 3], [1], [1, 4, 5, 6, 7, 8], [9, 3], [3], [3], [3], [3], [4]],
  [[2, 3, 4], [1], [1, 5], [1, 6, 7, 8, 9], [3], [4], [4], [4], [4]],
  [[2, 3], [1], [1, 4, 5, 6, 7], [3], [3], [3], [8, 9, 3], [7], [7]],
  [[2, 3, 4], [1], [1, 5, 6], [1, 7, 8, 9], [3], [3], [4], [4], [4]],
  [[2, 3], [1], [1, 4, 5, 6], [3], [3], [7, 8, 9, 3], [6], [6], [6]],
  [[2, 3, 4], [1, 5], [1, 6], [1, 7, 8, 9], [2], [3], [4], [4], [4]],
  [[2], [1, 3, 4], [2], [5, 6, 2, 7], [4], [4], [8, 9, 4], [7], [7]],
  [[2, 3], [1],

In [37]:
hold(10)


For F =  [[[2], [1, 3, 4, 5, 6, 7, 8, 9, 10], [2], [2], [2], [2], [2], [2], [2], [2]]]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 515 columns and 2568 nonzeros
Model fingerprint: 0x6a57b88b
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 3 rows and 502 columns
Presolve time: 0.00s
Presolved: 4 rows, 13 columns, 44 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.125000e+00   0.000000e+00      0s
       4    4.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.500000000e+00
Pesos asociados a los homomorfismos:  [0.5    0.     0.     0.

[[[[2], [1, 3, 4, 5, 6], [2], [2], [2], [7, 8, 9, 10, 2], [6], [6], [6], [6]],
  [[2, 3], [1], [1, 4, 5, 6, 7], [3], [3], [3], [8, 9, 10, 3], [7], [7], [7]],
  [[2], [1, 3, 4], [2], [5, 6, 7, 2, 8], [4], [4], [4], [9, 10, 4], [8], [8]],
  [[2, 3], [1], [1, 4, 5, 6, 7], [8, 3], [3], [3], [9, 10, 3], [4], [7], [7]],
  [[2, 3], [1], [1, 4, 5, 6, 7], [8, 3], [9, 3], [10, 3], [3], [4], [5], [6]],
  [[2, 3], [1, 4], [1, 5, 6, 7], [8, 9, 10, 2], [3], [3], [3], [4], [4], [4]],
  [[2, 3], [1, 4, 5], [1, 6, 7], [2], [8, 9, 10, 2], [3], [3], [5], [5], [5]],
  [[2, 3], [1, 4], [1, 5, 6, 7], [2], [3], [3], [8, 9, 10, 3], [7], [7], [7]],
  [[2, 3], [1], [1, 4, 5, 6], [3], [3], [7, 8, 9, 3], [10, 6], [6], [6], [7]],
  [[2, 3, 4], [1], [1, 5], [1, 6, 7, 8], [3], [4], [4], [9, 10, 4], [8], [8]],
  [[2, 3], [1], [1, 4, 5, 6], [7, 3], [3], [8, 9, 3], [4, 10], [6], [6], [7]],
  [[2, 3, 4], [1], [1, 5, 6], [1, 7, 8], [3], [3], [4], [9, 10, 4], [8], [8]],
  [[2, 3], [1], [1, 4, 5], [6, 3], [7, 8, 3], [4, 9,